# How to build an agent with physical properties?

In [2]:
from scenario import *
scen=Scenario()
cube=Cube()
cube.position=Vector3(0,0,2)
scen.add(cube)
while scen.t<10:
    scen.step(0.1)
    scen.render()